In [33]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 14: One-Time Pad ---</h2><p>In order to communicate securely with Santa while you're on this mission, you've been using a <a href="https://en.wikipedia.org/wiki/One-time_pad">one-time pad</a> that you <a href="https://en.wikipedia.org/wiki/Security_through_obscurity">generate</a> using a <span title="This also happens to be the plot of World War II.">pre-agreed algorithm</span>. Unfortunately, you've run out of keys in your one-time pad, and so you need to generate some more.</p>
<p>To generate keys, you first get a stream of random data by taking the <a href="https://en.wikipedia.org/wiki/MD5">MD5</a> of a pre-arranged <a href="https://en.wikipedia.org/wiki/Salt_(cryptography)">salt</a> (your puzzle input) and an increasing integer index (starting with <code>0</code>, and represented in decimal); the resulting MD5 hash should be represented as a string of <em>lowercase</em> hexadecimal digits.</p>
<p>However, not all of these MD5 hashes are <em>keys</em>, and you need <code>64</code> new keys for your one-time pad.  A hash is a key <em>only if</em>:</p>
<ul>
<li>It contains <em>three</em> of the same character in a row, like <code>777</code>. Only consider the first such triplet in a hash.</li>
<li>One of the next <code>1000</code> hashes in the stream contains that same character <em>five</em> times in a row, like <code>77777</code>.</li>
</ul>
<p>Considering future hashes for five-of-a-kind sequences does not cause those hashes to be skipped; instead, regardless of whether the current hash is a key, always resume testing for keys starting with the very next hash.</p>
<p>For example, if the pre-arranged salt is <code>abc</code>:</p>
<ul>
<li>The first index which produces a triple is <code>18</code>, because the MD5 hash of <code>abc18</code> contains <code>...cc38887a5...</code>. However, index <code>18</code> does not count as a key for your one-time pad, because none of the next thousand hashes (index <code>19</code> through index <code>1018</code>) contain <code>88888</code>.</li>
<li>The next index which produces a triple is <code>39</code>; the hash of <code>abc39</code> contains <code>eee</code>. It is also the first key: one of the next thousand hashes (the one at index 816) contains <code>eeeee</code>.</li>
<li>None of the next six triples are keys, but the one after that, at index <code>92</code>, is: it contains <code>999</code> and index <code>200</code> contains <code>99999</code>.</li>
<li>Eventually, index <code>22728</code> meets all of the criteria to generate the <code>64</code>th key.</li>
</ul>
<p>So, using our example salt of <code>abc</code>, index <code>22728</code> produces the <code>64</code>th key.</p>
<p>Given the actual salt in your puzzle input, <em>what index</em> produces your <code>64</code>th one-time pad key?</p>
</article>


In [34]:
from hashlib import md5
from itertools import islice


salt_example = "abc"


def otps(salt: str) -> str:
    three_in_a_row = re.compile(r"(\w)\1{2,}")
    md5s = []

    keys = []
    i = 0

    while len(keys) < 64:
        if len(md5s) - i < 1000:
            n = len(md5s)
            md5s += [
                md5((salt + f"{i}").encode()).hexdigest() for i in range(n, n + 1065)
            ]
        key = md5s[i]

        m = re.search(three_in_a_row, key)
        if m is not None and any(m[1] * 5 in k for k in islice(md5s, i + 1, i + 1001)):
            keys.append((i, key))

        i += 1

    return keys


otps(salt_example)[-1]

(22728, '26ccc731a8706e0c4f979aeb341871f0')

In [35]:
otps("jlmsuwbz")[-1]

(35186, 'cbd6d5c46cb652f90412cb1973222226')

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>35186</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Of course, in order to make this process <a href="https://en.wikipedia.org/wiki/MD5#Security">even more secure</a>, you've also implemented <a href="https://en.wikipedia.org/wiki/Key_stretching">key stretching</a>.</p>
<p>Key stretching forces attackers to spend more time generating hashes. Unfortunately, it forces everyone else to spend more time, too.</p>
<p>To implement key stretching, whenever you generate a hash, before you use it, you first find the MD5 hash of that hash, then the MD5 hash of <em>that</em> hash, and so on, a total of <em><code>2016</code> additional hashings</em>. Always use lowercase hexadecimal representations of hashes.</p>
<p>For example, to find the stretched hash for index <code>0</code> and salt <code>abc</code>:</p>
<ul>
<li>Find the MD5 hash of <code>abc0</code>: <code>577571be4de9dcce85a041ba0410f29f</code>.</li>
<li>Then, find the MD5 hash of that hash: <code>eec80a0c92dc8a0777c619d9bb51e910</code>.</li>
<li>Then, find the MD5 hash of that hash: <code>16062ce768787384c81fe17a7a60c7e3</code>.</li>
<li>...repeat many times...</li>
<li>Then, find the MD5 hash of that hash: <code>a107ff634856bb300138cac6568c0f24</code>.</li>
</ul>
<p>So, the stretched hash for index <code>0</code> in this situation is <code>a107ff...</code>. In the end, you find the original hash (one use of MD5), then find the hash-of-the-previous-hash <code>2016</code> times, for a total of <code>2017</code> uses of MD5.</p>
<p>The rest of the process remains the same, but now the keys are entirely different. Again for salt <code>abc</code>:</p>
<ul>
<li>The first triple (<code>222</code>, at index <code>5</code>) has no matching <code>22222</code> in the next thousand hashes.</li>
<li>The second triple (<code>eee</code>, at index <code>10</code>) hash a matching <code>eeeee</code> at index <code>89</code>, and so it is the first key.</li>
<li>Eventually, index <code>22551</code> produces the <code>64</code>th key (triple <code>fff</code> with matching <code>fffff</code> at index <code>22859</code>.</li>
</ul>
<p>Given the actual salt in your puzzle input and using <code>2016</code> extra MD5 calls of key stretching, <em>what index</em> now produces your <code>64</code>th one-time pad key?</p>
</article>

</main>


In [36]:
from hashlib import md5
from itertools import islice


salt_example = "abc"


def otps_key_stretching(salt: str) -> str:
    three_in_a_row = re.compile(r"(\w)\1{2,}")
    md5s = []

    keys = []
    i = 0

    while len(keys) < 64:
        if len(md5s) - i < 1000:
            n = len(md5s)
            md5s += [hash(salt, i) for i in range(n, n + 1065)]
        key = md5s[i]

        m = re.search(three_in_a_row, key)
        if m is not None and any(m[1] * 5 in k for k in islice(md5s, i + 1, i + 1001)):
            keys.append((i, key))

        i += 1

    return keys


def hash(salt, i):
    key = md5((salt + f"{i}").encode())
    for _ in range(2016):
        key = md5(key.hexdigest().encode())
    return key.hexdigest()


otps_key_stretching(salt_example)[-1]

(22551, '2df6e9378c3c53abed6d3508b6285fff')

In [37]:
otps_key_stretching("jlmsuwbz")[-1]

(22429, '922b7bd4af8e0dcc5a4ddd7eaef72569')

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>22429</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2016">return to your Advent calendar</a> and try another puzzle.</p>
<p>Your puzzle input was <code class="puzzle-input">jlmsuwbz</code>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22One%2DTime+Pad%22+%2D+Day+14+%2D+Advent+of+Code+2016&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F14&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22One%2DTime+Pad%22+%2D+Day+14+%2D+Advent+of+Code+2016+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F14';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
